Thought for a second


**Capítulo 2: Limpieza y Preparación de Datos**

Después de haber recorrido en el Capítulo 1 (01\_EDA) el universo de datos—sus orígenes, estructura, calidad y primeras revelaciones—llegamos al umbral de nuestro Capítulo 2: “Limpieza y Preparación de los Datos” (02\_clear). En esta nueva sección, nos sumergiremos en la transformación necesaria para convertir la materia prima analizada en un insumo confiable para el modelado predictivo.

En el notebook anterior, identificamos:

1. **Vacíos y valores atípicos** en la columna `onpromotion`, que, al exhibir cerca del 17 % de registros nulos, obligan a una estrategia de imputación clara.
2. **Huecos y duplicados** en la serie temporal de precios de crudo (`dcoilwtico`), con 43 valores faltantes y 220 entradas redundantes, lo que compromete la integridad de la variable macroeconómica más sensible para nuestros modelos.
3. **Oportunidades de mejora** en la alineación temporal de todos los datasets, así como la necesidad de homogeneizar formatos de fecha y tipos de dato para facilitar el procesamiento distribuido.

Por ello, en este Capítulo 2 procederemos a:

* **Reparar `train.csv`**: mediante Dask, leeremos en particiones controladas el conjunto de entrenamiento, rellenaremos todos los `NaN` de `onpromotion` con `False`, y generaremos un único CSV optimizado para GPU y carga distribuida.
* **Refrescar la serie de `oil.csv`**: descargaremos la fuente diaria de Dukascopy, convertiremos las marcas de tiempo a fechas limpias, seleccionaremos únicamente el cierre (`Close`) renombrado a `dcoilwtico`, y exportaremos un CSV definitivo sin huecos ni duplicados.

Con ello, garantizamos que los datos —libres de inconsistencia y preparados para el escalado paralelo— puedan alimentar sin fricción nuestra fase de ingeniería de características y modelado. Así, el Capítulo 2 no es un mero trámite: es el cimiento que sostendrá la precisión y la robustez de las predicciones de ventas diarias que persigue Corporación Favorita. Continuemos, pues, la historia de nuestro análisis hacia un entorno limpio y ordenado, listo para extraer conocimiento de forma confiable.


Bueno ahora que sabemosque preguntas podemos rsolver cosa que nos generaran insgiht beneficiosos para la empresa antes e hacer una einvestigacion exhaustiva primero tenemos que limpiar y arregla el set de dato par que sean aptos para una analiis 

hay varios problemas identificados que trataremos resolver aqui en el notebook:
primero la limpieza de eldaataframe train y el remplazon de nan en el daatafrane oil



## DataFrame `train.csv`

En este DataFrame se llevará a cabo el tratamiento y limpieza de la columna `onpromotion`.


In [ ]:
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import glob
import os

# Para que pandas no haga downcasting automático en el futuro
pd.set_option('future.no_silent_downcasting', True)

# Rutas de entrada y salida
INPUT_PATH   = "../data/raw/train.csv"
OUTPUT_DIR   = "../data/processed/parted_train"  # Directorio temporal con particiones CSV
FINAL_OUTPUT = "../data/processed/clear_train.csv"  # Archivo único final

# Carga del DataFrame con Dask (ajusta blocksize según tu memoria)
df = dd.read_csv(
    INPUT_PATH,
    assume_missing=True,
    blocksize="32MB",   # particiones más pequeñas para evitar OOM
    low_memory=False,
    dtype={"onpromotion": "boolean"}
)

# Rellenar NaNs en "onpromotion" y convertir a bool
filled = df.assign(
    onpromotion=df["onpromotion"].fillna(False).astype(bool)
)

# Exportar particiones a CSV en OUTPUT_DIR
os.makedirs(OUTPUT_DIR, exist_ok=True)
with ProgressBar():
    filled.to_csv(
        os.path.join(OUTPUT_DIR, "part-*.csv"),
        index=False,
        header=True
    )

# Unir particiones en un solo archivo CSV (concatenando sin duplicar encabezados)
csv_files = sorted(glob.glob(os.path.join(OUTPUT_DIR, "part-*.csv")))
with open(FINAL_OUTPUT, 'w', encoding='utf-8') as fout:
    header_written = False
    for fname in csv_files:
        with open(fname, 'r', encoding='utf-8') as fin:
            for i, line in enumerate(fin):
                if i == 0:
                    if not header_written:
                        fout.write(line)
                        header_written = True
                else:
                    fout.write(line)

print("Exportación completada en:", FINAL_OUTPUT)


[########################################] | 100% Completed | 482.92 s
Exportación completada en: ../data/processed/clear_train.csv


## Corrección del DataFrame descargado mediante Dukascopy

Se procederá a realizar el tratamiento del DataFrame descargado a través de Dukascopy, el cual contiene datos actualizados sobre el precio del petróleo.

Dado que eliminar estos datos podría implicar la pérdida de información valiosa y promediarlos podría inducir errores, se ha optado por utilizar datos completos obtenidos directamente desde el bróker Dukascopy.


> Se realizará una depuración del DataFrame (`oil_raw_Dukascopy.csv`) descargado a través del sitio web de Dukascopy, el cual contiene información detallada sobre el precio del petróleo.  
>  
> Durante este proceso, se eliminarán las columnas innecesarias y se corregirá el formato de la columna de fecha, ya que los datos descargados desde la página web suelen incluir información irrelevante.  
>  
> El objetivo es unificar la estructura del nuevo conjunto de datos con la del conjunto de datos original (`oil.csv`), asegurando que ambas fuentes tengan nombres de columnas consistentes y formatos homogéneos para facilitar su integración y análisis posterior.


In [4]:
import pandas as pd

# 1. Leer el CSV (ajusta 'ruta/al/archivo.csv' al nombre de tu fichero)
input_path = '../data/raw/oil_raw_Dukascopy.csv'
df = pd.read_csv(input_path, sep=',')

# 2. Convertir "Local time" a datetime y quedarnos solo con la fecha
#    Pandas detecta automáticamente el offset GMT-0500
df['date'] = pd.to_datetime(df['Local time']).dt.date

# 3. Seleccionar solo las columnas de interés
df = df[['date', 'Close']]

# 4. Renombrar "Close" a "dcoilwtico"
df = df.rename(columns={'Close': 'dcoilwtico'})

# 5. Exportar el resultado a la ruta que especifiques
#    Ajusta 'ruta/de/salida.csv' a la dirección deseada
output_path = '../data/processed/clear_oil_raw_Dukascopy.csv'
df.to_csv(output_path, index=False)


C:\Users\spinz\AppData\Local\Temp\ipykernel_26000\1321048040.py:9: UserWarning: Parsing dates in %d.%m.%Y %H:%M:%S.%f GMT%z format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date'] = pd.to_datetime(df['Local time']).dt.date
